# Running Bokeh on Jupyter Notebook
This demo shows off an interactive visualization using Bokeh for plotting, and Ipython interactors for widgets. The demo runs entirely inside the Ipython notebook, with no Bokeh server required.
The dropdown offers a choice of trig functions to plot, and the sliders control the frequency, amplitude, and phase.
To run, click on, Cell->Run All in the top menu, then scroll to the bottom and move the sliders.

In [1]:
import numpy as np
from ipywidgets import interact,widgets,interactive
from bokeh.io import push_notebook, output_notebook,show
from bokeh.plotting import figure
import pandas as pd
from bokeh.models.widgets import Slider, Select, TextInput

output_notebook()



Loading BokehJS ...

# Working with levers

In [2]:
SRECNA1 =pd.read_excel('SRECNA.xlsx')

In [3]:
#Index at 0,7,12,13,14
S=SRECNA1.loc[[0]]
S7=SRECNA1.loc[[7]]
S12=SRECNA1.loc[[12]]
S13=SRECNA1.loc[[13]]
S14=SRECNA1.loc[[14]]
OperatingRevenues=S.append([S7,S12,S13,S14])

In [5]:
revenueValues=OperatingRevenues.values[:,8:]
OperatingExpenses = SRECNA1[17:24]
expensesValues=OperatingExpenses.values[:,8:]
#Values of Operating expenses and revenues 
Values =np.vstack((revenueValues,expensesValues))



In [6]:
# Class for the operating levelers
class Operator:
    """An operator operates on the global set of data  scenario."""

    def __init__(self, section):
        """Create an operator with a given section with the given exit.

        name -- A string; the name of this Place.
        exit -- The Place reached by exiting this Place (may be None).
        """
        self.section = section
        #self.bees = []
        self.years_update=Scenario.shape[1]
        # Phase 1: Add an entrance to the exit
    def leveler(self, assumption):
        """Add an Insect to this Place.

        There can be at most one Ant in a Place, unless exactly one of them is
        a BodyguardAnt (Phase 4), in which case there can be two. If add_insect
        tries to add more Ants than is allowed, an assertion error is raised.

        There can be any number of Bees in a Place.
        """
        i=3
        while(i<self.years_update-1):    
            newEntry=Scenario[self.section,i]*(1.0+assumption)
            Scenario[self.section,i+1]=newEntry
            i+=1
        totalRevs=np.sum(Scenario[0:5,:],axis=0)
        totalExps=np.sum(Scenario[5:,:],axis=0)
        new_data=totalRevs-totalExps
        return new_data

In [7]:
# We have a numpy matrix of values, dimensions 12X15 of operating revenues and expenses that we need to index with yearIndex 
# for columns and we index the row with sectionIndex that selects that operating revenue or expense. 
#Finally once we have indexed the desired entry with the year and section we can apply the assumption percentage
def OperatingMargin(sectionIndex,assumption):
    newEntry=Values[3,sectionIndex]
    newEntry=newEntry*(1.0+assumption)
    Scenario=Values[:]
    Scenario[3,sectionIndex]=newEntry
    totalRevs=np.sum(Scenario[0:5,:],axis=0)
    totalExps=np.sum(Scenario[5:,:],axis=0)
    return totalRevs-totalExps


In [14]:
Scenario=np.copy(Values)
years=np.linspace(2014,2020,7)
y=OperatingMargin(0,0)
model = figure(title="Budget Model", plot_height=600, plot_width=800) 
r = model.line(years,y, color="#2222aa", line_width=2)
#Change background color
model.background_fill_color = "white"
# change just some things about the x-axes
model.xaxis.axis_label = "Years"
model.xaxis.axis_line_width = 2
model.xaxis.axis_line_color = "Black"

# change just some things about the y-axes

model.yaxis.axis_label = "Operating Margin"
model.yaxis.major_label_text_color = "Black"
model.yaxis.major_label_orientation = "vertical"

# change just some things about the x-grid
model.xgrid.grid_line_color = "Black"
model.xgrid.grid_line_alpha = 0.3
# change just some things about the y-grid
model.ygrid.grid_line_alpha = 0.3
model.ygrid.grid_line_color = "Black"
#Change the outline of the plot box
model.outline_line_width = 7
model.outline_line_alpha = 0.3
model.outline_line_color = "navy"

#Interacting widgets for Revenues
def tuition(assume=0):
    selection=Operator(section=0)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def grants(assume=0):
    selection=Operator(section=1)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()

def educational(assume=0):
    selection=Operator(section=2)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def auxiliary(assume=0):
    selection=Operator(section=3)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def otherr(assume=0):
    selection=Operator(section=4)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
#Interacting widgets for expenses

def salaries(assume=0):
    selection=Operator(section=5)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def benefits(assume=0):
    selection=Operator(section=6)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def scholarships(assume=0):
    selection=Operator(section=7)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def utilities(assume=0):
    selection=Operator(section=8)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def supplies(assume=0):
    selection=Operator(section=9)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
    
def depreciation(assume=0):
    selection=Operator(section=10)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()

def othere(assume=0):
    selection=Operator(section=11)
    new_data=selection.leveler(assumption=assume)
    r.data_source.data['y'] = new_data
    push_notebook()
show(model,notebook_handle=True)

#Revenues Levelers
R1=interactive(tuition, assume=widgets.FloatSlider(description="Tuition",min=0, max=0.5,step=0.01,continuous_update=False))
R2=interactive(grants, assume=widgets.FloatSlider(description="Grants",min=0, max=0.5,step=0.01,continuous_update=False))
R3=interactive(educational, assume=widgets.FloatSlider(description="Educational",min=0, max=0.5,step=0.01,continuous_update=False))
R4=interactive(auxiliary, assume=widgets.FloatSlider(description="Auxiliary",min=0, max=0.5,step=0.01,continuous_update=False))
R5=interactive(otherr, assume=widgets.FloatSlider(description="Other",min=0, max=0.5,step=0.01,continuous_update=False))

#Expenses Levelers
E1=interactive(salaries, assume=widgets.FloatSlider(description="Salaries",min=0, max=0.5,step=0.01,continuous_update=False))
E2=interactive(benefits, assume=widgets.FloatSlider(description="Benefits",min=0, max=0.5,step=0.01,continuous_update=False))
E3=interactive(scholarships, assume=widgets.FloatSlider(description="Scholarships",min=0, max=0.5,step=0.01,continuous_update=False))
E4=interactive(utilities, assume=widgets.FloatSlider(description="Utilities",min=0, max=0.5,step=0.01,continuous_update=False))
E5=interactive(supplies, assume=widgets.FloatSlider(description="Supplies",min=0, max=0.5,step=0.01,continuous_update=False))
E6=interactive(depreciation, assume=widgets.FloatSlider(description="Depreciation",min=0, max=0.5,step=0.01,continuous_update=False))
E7=interactive(othere, assume=widgets.FloatSlider(description="Other",min=0, max=0.5,step=0.01,continuous_update=False))



#list = ['P0', 'P1', 'P2', 'P3', 'P4']
#children = [widgets.Text(description=name) for name in list]
revenues=[R1,R2,R3,R4,R5,]
expenses=[E1,E2,E3,E4,E5,E6,E7]
v1=widgets.VBox(revenues)
v2=widgets.VBox(expenses)
children=[v1,v2]
tab = widgets.Tab(children=children)
tab.set_title(0,"Revenues")
tab.set_title(1,"Expenses")




In [15]:
tab


# New Data 

In [30]:
SRECNA2 =pd.read_excel('SISD.xlsx')
SRECNA3=SRECNA2.dropna(axis=0,how='all')  #Drops rows with all NaN values

In [34]:

Indexes=list(SRECNA3.index.values)
SRECNA3=SRECNA3.drop([Indexes[8],Indexes[20],Indexes[19],Indexes[18],Indexes[17],Indexes[16],Indexes[15]],axis=0)
newValues=SRECNA3.values[:,1:]
In=list(SRECNA3.index.values)
sorted(widgets.Widget.widget_types)